In [1]:
import gurobipy as gp
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math

## Generazione dataset

Prima genera tutti i CC e gli SP che non appartengono a cluster, sono distribuiti uniformemente nella mappa.
Poi seleziona 10 CC che saranno il centro dei cluster, per ognuno di essi genera le coordinate degli SP (uniformemente), nella regione inclusa fra +- 3 punti di distanza. Tutti i cluster hanno stesso numero di SP

In [2]:
#la funzione prende in input: k dimensioni mappa, N_SP numero di source points, n_cluster numero di cluster 
#e cluster ratio percentuale di punti facenti parte di un cluster
#ritorna in output vettori numpy contenenti le coordinate dei punti
def data_generation(k, n_SP, n_CC, n_cluster, cluster_ratio):
    
    #definisco coordinate di tutti i CC e degli SP non facenti parte dei cluster
    P=np.random.randint(0, k+1, size=(n_CC, 2))  #genera il set dei CC
    S_no_cluster=np.random.randint(0, k+1, size=(n_SP-int(cluster_ratio*n_SP), 2))  #genera il set dei SP non facenti parte di un cluster

    #genero coordinate SP rimanenti
    cluster_index=random.sample(range(0,n_CC), int(n_CC*cluster_ratio)) #seleziona l'indice dei CC a caso che diventeranno i centri dei cluster
    S_cluster_list=[]  #creo lista vuota da popolare con i numpy array e poi concatenarli TROVA MODO PIU EFFICIENTE

    #per ognuno dei cluster genero casualmente uniformemente le coordinate (entro i 36 punti adiacenti) di 5 SP
    for i in cluster_index:
        #voglio considerare il contorno dei 36 vicini, quindi indici (i-3, j-3) (i+3, j+3)
        xmin=P[i][0]-3
        xmax=P[i][0]+3

        ymin=P[i][1]-3
        ymax=P[i][1]+3
    
        #considera ogni cluster con stesso numero di SP
        S_cluster_list.append(np.random.randint(low=[xmin, ymin], high=[xmax, ymax], size=(round(cluster_ratio*n_SP/n_cluster), 2)))

    #mette assieme il tutto
    S_cluster=np.concatenate(S_cluster_list) #crea array unico per i cluster
    S=np.concatenate((S_cluster, S_no_cluster))

    #genera posizione della fabbrica biodiesel
    biodiesel=np.random.randint(0, k+1, size=(1, 2))
    
    return P, S, biodiesel

In [3]:
#funzione che date le coordinate di due punti (in array 2d numpy) ritorna la distanza euclidea arrotondata in integer
def distance(a, b):
    dist=round(math.sqrt((a[0]-b[0])**2+(a[1]-b[1])**2))
    return dist

In [4]:
k=1000  #dimensioni della mappa (prima test con k=1000, poi varialo per studio domansionale)

n_SP=100   #number of source points
n_CC=20 #number of collection centers
n_cluster=int(0.5*n_CC)  #number of clusters
cluster_ratio=0.5  #how many of the total nodes (SPs and CCs) fall within a cluster

#RICORDATI DI COMMENTARE IL SEED PER MISURARE PERFORMANCE
np.random.seed(seed=420) #fissa il seed per riproducibilità nei test

P, S, biodiesel=data_generation(k, n_SP, n_CC, n_cluster, cluster_ratio)


#definizione e generazione delle altre costanti
alpha=1  #costo per unità di distanza
V=5  #numero di veicoli MESSO A CASO, NEL PAPER NON TROVO QUANTO DOVREBBE ESSERE
Q=24  #numero di cestini su ogni veicolo
B=50  #capacità, in litri, dei bins sui veicoli
b=10  #costo di un bin
max_distance=300  #distanza massima fra un SP e un CC
R=1800 #lungezza massima percorso per veicolo, se distanza fra 0,0 e 1000,1000 è 1414 allora 1800 non è sufficiente nel caso il centrp biodiesel venga generato in un angolo?
P0=np.concatenate((P, biodiesel)) #set posizione biodiesel+CC (serve in calcoli veicoli)
f=np.random.randint(50, 150+1, size=(n_CC))  #costo per aprire il CC i
d_s=np.random.randint(1, 20+1, size=(len(S)))  #produzione settimanale in litri di olio del SP i
C=np.random.randint(1, 5+1, size=(n_CC))  #numero di Bins nel CC i


#P_s è il set dei CC entro la distanza di copertura dal SP s
#quindi per ogni SP creo lista P_s contente indice dei CC compatibili, poi salvo in lista di liste il tutto
P_s_list=[]  #inizializzo lista che conterrà i vari P_s
for s in range(len(S)):
    P_s=[]  #inizializzo P_s
    for i in range(len(P)): 
        if (distance(S[s], P[i])<300):   #controlla se la distanza è minore di 300
            P_s.append(i)  #se distanza e minore di 300 salva l'indice del CC compatibile
    P_s_list.append(P_s)  #aggiunge la lista alla lista totale


#crea matrice delle distanze fra elementi di P0
c0=np.empty([len(P0), len(P0)])
for i in range(len(P0)):
    for j in range(len(P0)):
        c0[i,j]=distance(P0[i], P0[j])

#crea matrice delle distanze fra elementi di S e P
cs=np.empty([len(S), len(P)])
for i in range(len(S)):
    for j in range(len(P)):
        cs[i,j]=distance(S[i], P[j])        

## Constructive heuristic

Fase 1 focus su CC location, SP assignment e bin allocation.
Fase 2 si occupa del percorso dei veicoli.

in fase 1 definisco attractiveness score per trovare set CC candidati (funzione first assignment) per poi assegnare gli SP e i bin ai CC selezionati (funzione APF).

in fase 2 si genera il percorso utilizzando algoritmo modificato di Clarke-Wright che tiene conto anche delle maximum route constraints (funzione clarke-wright).

In [5]:
#riduce il numero di possibili CC da aprire
def first_assignment(omega, n):
        P_hat=[] #set dei CC candidati
        S_to_exc=[] #indice degli SP con soluzione unica
        #prima identifico gli SP che possono essere connessi a un solo CC, che dovrà per forza essere aperto e quindi aggiunto a P_hat
        for i in range(len(P_s_list)):
            if (len(P_s_list[i])==1 and P_s_list[i][0] not in P_hat): #controlla se lista SP ha un solo CC vicino e se il CC non è già stato aggiunto a P_hat
                P_hat.append(P_s_list[i][0])
                S_to_exc.append(i)
        
        S_ch_set=set(S_set)-set(S_to_exc) #eslude SP con un unico CC vicino
        CC_ch_set=set(P_set)-set(P_hat) #esclude i CC assegnati all'SP di cui sopra

        #calcolo attractiveness
        A=np.empty(shape=(n_CC))
        #omega, n sono parametri da settare successivamente; intanto sono fissati per test

        #set dei closest CC to CC_i
        CC_closest_list=[]  #inizializzo lista che conterrà i vari set dei closest CC
        for index in range(n_CC):
            CC_closest_list.append([i[0] for i in sorted(enumerate(c0[index,:]), key=lambda x:x[1])]) #ritorna lista indici ordinati per distanza 
            #il primo elemento di ogni lista di CC_closest_list è il CC stesso, quindi va ignorato    

        #per costruire P_hat:
        while len(S_ch_set)!=0:  #finchè non vengono assegnati tutti gli SP
            for i in range(n_CC): #calcola attractiveness
                if i in P_hat:
                    A[i]=0  #per evitare di selezionare CC già in P_hat
                else:    
                    A[i]=min(C[i], sum(d_s[s] for s in S_set if i in P_s_list[s]))/(omega*c0[i,len(P0)-1]+(1-omega)*sum(c0[i][j] for j in CC_closest_list[i][1:n+1])/n)
            #seleziona attractiveness massima
            temp_max_index=max(enumerate(A[:]), key=lambda x:x[1])[0]  #trova indice e (valore massimo di A)      
            P_hat.append(temp_max_index) 
    
            #Trova gli SP vicini al CC con attractiveness massima
            S_accessible=[]
            for s in S_ch_set: 
                if temp_max_index in P_s_list[s]:
                    S_accessible.append(s)
            #calcola la capacità residua dei bin per assegnare gli SP più vicini al CC:
            C_res=B*C[temp_max_index]
            while len(S_accessible)!=0:
                #trova indice di SP più vicino    
                s_star=list(set(np.where(cs==min(cs[list(S_accessible) ,temp_max_index]))[0]).intersection(S_accessible))[0]  
                S_accessible=set(S_accessible)-set([s_star])
                if C_res>=d_s[s_star]: #controlla se SP produce meno olio della capacità residua
                    C_res=C_res-d_s[s_star]
                    S_ch_set=set(S_ch_set)-set([s_star]) #esclude SP appena assegnati
        
        #costruisco soluzione con AP-S o AP-F
        
        #P_hat_s è il set dei CC entro la distanza di copertura dal SP s
        #quindi per ogni SP creo lista P_s contente indice dei CC compatibili, poi salvo in lista di liste il tutto
        P_hat_s_list=[]  #inizializzo lista che conterrà i vari P_hat_s
        for s in range(len(S)):
            P_hat_s=[]  #inizializzo P_hat_s
            for i in (P_hat): 
                if (distance(S[s], P[i])<300):   #controlla se la distanza è minore di 300
                    P_hat_s.append(i)  #se distanza e minore di 300 salva l'indice del CC compatibile
            P_hat_s_list.append(P_hat_s)  #aggiunge la lista alla lista totale
            
        return(P_hat, P_hat_s_list)  #ritorno i possibili CC da aprire e gli SP compatibili

In [6]:
#definisce il modello gurobi per assegnare gli SP
def APf(P_hat, P_hat_s_list):
        #define the AP-S (and AP-F)
        AP=gp.Model() #crea il modello

        #decision variables
        #1 se il CC i viene aperto, 0 altrimenti
        x_i=AP.addVars([(i) for i in P_hat], vtype=gp.GRB.BINARY) 

        #1 se il SP s viene assegnato al CC i, 0 altrimenti.
        z_si=AP.addVars([(s, i) for s in range(len(S)) for i in P_hat_s_list[s]], vtype=gp.GRB.BINARY) 

        #numero di bins in CC i
        y_i=AP.addVars([(i) for i in P_hat], lb=0, vtype=gp.GRB.INTEGER) 


        #objective function and constraints
        #20)objective function
        AP.setObjective(gp.quicksum(f[i]*x_i[i] for i in P_hat)+
                        b*gp.quicksum(y_i[i] for i in P_hat)
                        )

        #21)nel paper è sbagliato, z_si[s, i] i dipende da s e non esiste per tutti elementi P_hat
        #definisco allo stesso modo del constraint 2 del MIP
        AP.addConstrs(gp.quicksum(d_s[s]*z_si[s, i] for s in range(len(S)) if i in P_hat_s_list[s])<=
                      B*C[i]*x_i[i]
                      for i in P_hat 
                      )

        #22)
        AP.addConstrs(z_si[s,i]<=
                      x_i[i]
                      for s in S_set for i in P_hat_s_list[s] 
                      )    

        #23) 
        AP.addConstrs(gp.quicksum(z_si[s,i] for i in P_hat_s_list[s])==1 
                      for s in range(len(S))  
                      )   

        #24)nel paper è sbagliato, z_si[s, i] i dipende da s e non esiste per tutti elementi P_hat
        #definisco allo stesso modo del constraint 2 del MIP
        AP.addConstrs(gp.quicksum(d_s[s]*z_si[s, i] for s in range(len(S)) if i in P_hat_s_list[s])<=
                      B*y_i[i]
                      for i in P_hat 
                      )

        #constraint per passare a AP-F (AP-F forza tutti i CC trovati ad essere aperti)
        AP.addConstrs(x_i[i]==1 
                      for i in P_hat  
                      )   
        #update del modello per aggiunger la funzione obbiettivo e i constraints       
        AP.update()
        #25-27) sono gia definiti implicitamente nella inizializzazione delle decision variables    
        return AP, x_i, z_si, y_i #ritorna il modello e le variabili

In [7]:
#versione modificata di algoritmo di Clarke-Wright che tiene conto anche della durata massima del percorso.
def clarke_wright(opened_CC, number_bins):
        #per estrarre i CC aperti
        P_open=[]
        for i in P_hat:
            if (opened_CC[i]==1):
                P_open.append(P[i])
        P_open=np.concatenate([P_open]) #mette il tutto in un unico np.array

        #per estrarre il numero di bins
        bins=[]
        for i in P_hat:
            if (opened_CC[i]==1):
                bins.append(number_bins[i])
        bins=np.concatenate([bins]) #mette il tutto in un unico np.array

        #versione modificata di algoritmo Clarke-Wright savings che tiene anche conto della lunghezza massima di un percorso
        #calcola la matrice dei savings
        savings=np.empty(shape=(len(P_open),len(P_open)))
        for i in range(len(P_open)):
            for j in range(len(P_open)):
                if i==j:
                    savings[i,j]=0
                else:    
                    savings[i,j]=distance(biodiesel[0], P_open[i])+distance(biodiesel[0], P_open[j])-distance(P_open[i], P_open[j])

        #ordina in ordine decrescente i savings (considerando savings[i,j]=savings[j,i])
        links=[]
        for i in range(len(P_open)):
            for j in range(i+1, len(P_open)):
                links.append([savings[i,j], [i,j]])
        links=sorted(links, key=lambda x: x[0], reverse=True)  #ordina i savings e il link in ordine decrescente
        links=[row[1] for row in links] #ritorna solo i links del percorso

        routes = [[i] for i in range(len(P_open))]
        for i in [row[0] for row in links]:
            for j in [row[1] for row in links]:
                route_i, route_j = None, None
                #Find the routes containing CC i and j
                for route in routes: 
                    if i in route:
                        route_i = route
                    if j in route:
                        route_j = route
                    
                    #If i and j are in different routes and merging them does not violate capacity and duration constraints
                    if (route_i != None) and (route_j != None):
                        #calcola lunghezza percorso
                        totale_i=0
                        for o in range(len(P_open[route_i])-1):
                            totale_i=distance(P_open[route_i][o], P_open[route_i][o+1])
                        totale_i+=distance(P_open[route_i][0],biodiesel[0])+distance(P_open[route_i][len(route_i)-1],biodiesel[0]) #add the distance from the biodiesel facility
                    
                        totale_j=0
                        for o in range(len(P_open[route_j])-1):
                            totale_j=distance(P_open[route_j][o], P_open[route_j][o+1])
                        totale_j+=distance(P_open[route_j][0],biodiesel[0])+distance(P_open[route_j][len(route_j)-1],biodiesel[0]) #add the distance from the biodiesel facility                        
                        
                        try: #errore che non so risolvere, avviene in una sola iterazione,interromperebbe esecuzione, ma si arriva a soluzione lo stesso
                            if (route_i != route_j) and (sum(bins[route_i]) + sum(bins[route_j]) <= Q) and (totale_i+totale_j)<=1800:
                                route_i_index = routes.index(route_i)
                                route_j_index = routes.index(route_j)
                                #Merge the routes
                                routes[route_i_index] += routes[route_j_index]
                                #Remove the second route
                                routes.pop(route_j_index)
                        except: 
                            continue
        for i in range(len(routes)):
            routes[i].insert(0,len(P0)) #aggiunge il centro biodiesel a inizio route
            routes[i].append(len(P0))  #aggiunge il centro biodiesel a fine route
        
        
        return routes #ritorna le routes sub-ottimali iniziali

In [8]:
#crea i set degli indici
P_set=range(0, n_CC) #CC
S_set=range(0, n_SP) #SP
P0_set=range(0, n_CC+1)  #CC+centro biodiesel (ultimo elemento)
V_set=range(0, V)  #veicoli


beta=[] #salva le soluzioni appena trovate in lista. 
#algoritmo 1
for omega in np.linspace(0,1,10):
    for n in range(1, math.ceil(n_CC/2)): #nel paper n parte da 0, ma in calcolo attractiveness è presente una divisione per n
         
        P_hat, P_hat_s_list=first_assignment(omega, n)   #Genera soluzione iniziale 
        

        AP, x_i, z_si, y_i=APf(P_hat, P_hat_s_list)  #Genera il modello gurobi da ottimizzare
        AP.modelSense = gp.GRB.MINIMIZE
        AP.optimize()  #ottimizza il modelo gurobi
        
        #accesso soluzioni
        opened_CC=AP.getAttr('x', x_i)
        connected_SP=AP.getAttr('x', z_si)
        number_bins=AP.getAttr('x', y_i)

        #usa le soluzioni appena trovate per costruire delle routes sub-ottimali 
        routes=clarke_wright(opened_CC, number_bins)
        #Gli indici della route in beta corrispondon a ordine degli opened_CC in beta, in VNS bisogna convertirli a indici di P/P0    
        #per estrarre i CC aperti
        P_open=[]
        for i in opened_CC.keys():
            if (opened_CC[i]==1):
                P_open.append([i])
        P_open=np.concatenate([P_open]) #mette il tutto in un unico np.array
        #per convertire gli elementi delle route a indici del CC
        for i in range(len(routes)): #seleziona route
            for j in range(1, len(routes[i])-1): #cambia gli indici interni, estremi corrispondono già al depot
                routes[i][j]=P_open[routes[i][j]][0]
        #aggiunge a lista i risultati trovati per i valori correnti di omega e n
        beta.append([opened_CC, connected_SP, number_bins, routes]) 
        AP.dispose()   #resetta il modello

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-26
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD Ryzen 3 3250U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 569 rows, 453 columns and 2153 nonzeros
Model fingerprint: 0xa396c039
Variable types: 0 continuous, 453 integer (437 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1925.0000000
Presolve removed 444 rows and 19 columns
Presolve time: 0.00s
Presolved: 125 rows, 434 columns, 1216 nonzeros
Variable types: 0 continuous, 434 integer (418 binary)

Root relaxation: objective 1.864400e+03, 149 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl


     0     0 1864.40000    0    3 1935.00000 1864.40000  3.65%     -    0s
H    0     0                    1885.0000000 1864.40000  1.09%     -    0s
     0     0 1864.40000    0    9 1885.00000 1864.40000  1.09%     -    0s
H    0     0                    1875.0000000 1864.40000  0.57%     -    0s
     0     0 1864.40000    0   12 1875.00000 1864.40000  0.57%     -    0s
     0     0 1864.40000    0    3 1875.00000 1864.40000  0.57%     -    0s
     0     0 1864.40000    0    3 1875.00000 1864.40000  0.57%     -    0s
     0     0 1864.40000    0    6 1875.00000 1864.40000  0.57%     -    0s
     0     0 1864.40000    0    8 1875.00000 1864.40000  0.57%     -    0s
     0     0 1864.40000    0    8 1875.00000 1864.40000  0.57%     -    0s
     0     2 1864.40000    0    8 1875.00000 1864.40000  0.57%     -    0s
H  846   615                    1865.0000000 1864.40000  0.03%   3.8    0s

Cutting planes:
  Gomory: 1
  Cover: 1
  Implied bound: 5

Explored 846 nodes (5375 simplex iterat


Optimal solution found (tolerance 1.00e-04)
Best objective 1.723000000000e+03, best bound 1.723000000000e+03, gap 0.0000%
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD Ryzen 3 3250U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 533 rows, 418 columns and 1985 nonzeros
Model fingerprint: 0x2bbbeae2
Variable types: 0 continuous, 418 integer (403 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1813.0000000
Presolve removed 410 rows and 18 columns
Presolve time: 0.00s
Presolved: 123 rows, 400 columns, 1116 nonzeros
Variable types: 0 continuous, 400 integer (385 binary)
Found heuristic solution: objective 1803.0000000

Root relaxation: objective 1.722400e+03, 154 iterations, 0.00 seconds (0.00 work units

     0     0 1864.40000    0   13 1875.00000 1864.40000  0.57%     -    0s
     0     0 1864.40000    0   13 1875.00000 1864.40000  0.57%     -    0s
     0     0 1864.40000    0    3 1875.00000 1864.40000  0.57%     -    0s
     0     0 1864.40000    0    3 1875.00000 1864.40000  0.57%     -    0s
     0     0 1864.40000    0    8 1875.00000 1864.40000  0.57%     -    0s
     0     0 1864.40000    0    9 1875.00000 1864.40000  0.57%     -    0s
     0     0 1864.40000    0    3 1875.00000 1864.40000  0.57%     -    0s
     0     0 1864.40000    0    3 1875.00000 1864.40000  0.57%     -    0s
     0     2 1864.40000    0    3 1875.00000 1864.40000  0.57%     -    0s
*  193   179              63    1865.0000000 1864.40000  0.03%   4.7    0s

Cutting planes:
  Implied bound: 3
  MIR: 4
  StrongCG: 5
  Zero half: 1

Explored 215 nodes (1560 simplex iterations) in 0.16 seconds (0.07 work units)
Thread count was 4 (of 4 available processors)

Solution count 4: 1865 1875 1885 1925 

Optimal 


Optimal solution found (tolerance 1.00e-04)
Best objective 1.865000000000e+03, best bound 1.865000000000e+03, gap 0.0000%
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD Ryzen 3 3250U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 550 rows, 435 columns and 2070 nonzeros
Model fingerprint: 0xea408ed5
Variable types: 0 continuous, 435 integer (420 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1787.0000000
Presolve removed 438 rows and 33 columns
Presolve time: 0.00s
Presolved: 112 rows, 402 columns, 1137 nonzeros
Variable types: 0 continuous, 402 integer (383 binary)

Root relaxation: objective 1.716400e+03, 134 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Object

...

  Gomory: 1
  Cover: 1
  Implied bound: 2
  MIR: 3
  StrongCG: 3

Explored 29 nodes (1187 simplex iterations) in 0.16 seconds (0.03 work units)
Thread count was 4 (of 4 available processors)

Solution count 4: 1526 1536 1576 1596 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.526000000000e+03, best bound 1.526000000000e+03, gap 0.0000%
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD Ryzen 3 3250U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 496 rows, 383 columns and 1824 nonzeros
Model fingerprint: 0xdea49885
Variable types: 0 continuous, 383 integer (370 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1596.0000000
Presolve removed 390 rows and 30 columns
Presolve time: 0.00s
Presolve


Explored 1 nodes (251 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 3: 1596 1656 1666 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.596000000000e+03, best bound 1.596000000000e+03, gap 0.0000%
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD Ryzen 3 3250U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 535 rows, 421 columns and 2007 nonzeros
Model fingerprint: 0x050772f2
Variable types: 0 continuous, 421 integer (407 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1666.0000000
Presolve removed 427 rows and 31 columns
Presolve time: 0.00s
Presolved: 108 rows, 390 columns, 1117 nonzeros
Variable types: 0 continuous, 39

Model fingerprint: 0x050772f2
Variable types: 0 continuous, 421 integer (407 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1666.0000000
Presolve removed 427 rows and 31 columns
Presolve time: 0.00s
Presolved: 108 rows, 390 columns, 1117 nonzeros
Variable types: 0 continuous, 390 integer (376 binary)
Found heuristic solution: objective 1656.0000000

Root relaxation: objective 1.595400e+03, 124 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1595.40000    0    2 1656.00000 1595.40000  3.66%     -    0s
H    0     0                    1596.0000000 1595.40000  0.04%     -    0s
     0     0 1595.40000    0    2 1596.00000 1595.40000  0.04%     -    0s

Explored 1 nodes (251 sim


Root relaxation: objective 1.595400e+03, 124 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1595.40000    0    2 1656.00000 1595.40000  3.66%     -    0s
H    0     0                    1596.0000000 1595.40000  0.04%     -    0s
     0     0 1595.40000    0    2 1596.00000 1595.40000  0.04%     -    0s

Explored 1 nodes (251 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 3: 1596 1656 1666 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.596000000000e+03, best bound 1.596000000000e+03, gap 0.0000%
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD Ryzen 3 3250U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 535 rows, 

Thread count was 4 (of 4 available processors)

Solution count 3: 1596 1656 1666 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.596000000000e+03, best bound 1.596000000000e+03, gap 0.0000%
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD Ryzen 3 3250U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 535 rows, 421 columns and 2007 nonzeros
Model fingerprint: 0x050772f2
Variable types: 0 continuous, 421 integer (407 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1666.0000000
Presolve removed 427 rows and 31 columns
Presolve time: 0.00s
Presolved: 108 rows, 390 columns, 1117 nonzeros
Variable types: 0 continuous, 390 integer (376 binary)
Found heuristic solution: objective 1656.0000000

Root

## VNS

Implementazione di una variable neighborhood search utilizzando le soluzioni appena trovate con constructive heuristic.

La VNS utilizza quattro diverse operazioni:

-)inserisce un segmento di una route in un altra route (insert)

-)scambia segementi di due route(swap)

-)rimuove un segmento di CCs da una route e aggiunge un numero di CCs non aperti alla route (insert & remove)

-)rimuove un CC dalla route (Remove)

In [216]:
import copy
#estrae le soluzioni come tupledict
opened_CC=copy.deepcopy(beta[0][0]) #CC aperti
connected_SP=copy.deepcopy(beta[0][1]) #connession CC-SP
number_bins=copy.deepcopy(beta[0][2]) #numero di bins in CC
#route è già un np.array
vehicle_route=copy.deepcopy(beta[0][3]) #route

P_open=[]
for i in opened_CC.keys():
    if (opened_CC[i]==1):
        P_open.append([i][0])

Implementazione delle 4 operazioni, per ora funzionano solo con segmenti di lunghezza 1 e aggiungendo togliendo un solo CC alla volta. eventualmente vanno aggiunte funzioni per applicare operazioni a segmenti di diversa lunghezza.

In [188]:
#sposta un segmento di lunghezza 1 in altra route
def insertion1(vehicle_route):
    i,j=random.sample(range(1, len(vehicle_route)-1),2) #seleziona due route a caso 
    k=random.sample(range(1, len(vehicle_route[i])-1),1)[0] #seleziona segmento route1 a caso
    l=random.sample(range(1, len(vehicle_route[j])-1),1)[0] #seleziona segmento route2 a caso
    vehicle_route[i].insert(k, #posizione in cui inserire segmento
                            vehicle_route[j][l]  #segmento da inserire
                            ) 
    vehicle_route[j].pop(l) #elimina il segmento spostato dalla route
    if (len(vehicle_route[j])==2): #elimina la route se non ci sono più CC
        vehicle_route.pop(j)

In [189]:
#scambia segmento di lunghezza 1 fra due route
def swap(vehicle_route):
    i,j=random.sample(range(1, len(vehicle_route)-1),2) #seleziona due route a caso 
    k=random.sample(range(1, len(vehicle_route[i])-1),1)[0] #seleziona segmento route1 a caso
    l=random.sample(range(1, len(vehicle_route[j])-1),1)[0] #seleziona segmento route2 a caso

    vehicle_route[i][k], vehicle_route[j][l]= vehicle_route[j][l], vehicle_route[i][k] #scambia i segmenti

In [223]:
#rimuove un CC da una route
def remove(vehicle_route):
    i=random.sample(range(1, len(vehicle_route)-1),1)[0] #seleziona route a caso 
    k=random.sample(range(1, len(vehicle_route[i])-1),1)[0] #seleziona segmento route1 a caso
    
    vehicle_route[i].pop(k) #elimina il segmento di CCs
    if (len(vehicle_route[i])==2): #elimina la route se non ci sono più CC
        vehicle_route.pop(i)

In [224]:
#rimuove segmento di CCs dalla route e aggiunge un CC chiuso a route 
def remove_add(vehicle_route, P_open, P_set):
    i=random.sample(range(1, len(vehicle_route)-1),1)[0] #seleziona route a caso 
    k=random.sample(range(1, len(vehicle_route[i])-1),1)[0] #seleziona segmento route1 a caso
    
    vehicle_route[i].pop(k) #elimina il segmento di CCs
    
    n=random.sample(list(set(P_set)-set(P_open)), 1) #sceglie CC non ancora aperto 
    
    vehicle_route[i].insert(k, #sposizione in cui inserire segmento
                            n  #segmento da inserire
                            ) 